In [ ]:
!pip install yellowbrick
!pip install xgboost
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler, OrdinalEncoder, PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score, root_mean_squared_error
import statsmodels.api as sm 
from sklearn.feature_selection import RFE
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from scipy.stats import randint, uniform
from yellowbrick.regressor import ResidualsPlot
from xgboost import XGBRegressor
from sklearn.decomposition import PCA
from sklearn.svm import SVR

In [ ]:
data= pd.read_excel("./Flight_Fare.xlsx")

# Exploratory Data Analysis

In [ ]:
data.describe()

In [ ]:
data.head()

In [ ]:
data.tail()

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
data.isnull().sum()

In [ ]:
data.isna().sum()

In [ ]:
data['Additional_Info'].value_counts()

In [ ]:
data['Airline'].value_counts()

In [ ]:
data['Source'].value_counts()

In [ ]:
data['Destination'].value_counts()

In [ ]:
data['Total_Stops'].value_counts()

In [ ]:
data[data['Airline'] == 'Jet Airways Business']['Additional_Info'].value_counts()

In [ ]:
data.loc[data['Airline'] == "Multiple carriers Premium economy", 'Additional_Info'].value_counts()

In [ ]:
data.loc[data['Airline'] == 'Vistara Premium economy', 'Additional_Info'].value_counts()

In [ ]:
# Price Distribution
plt.figure(figsize=(8, 5))
sns.histplot(data['Price'], bins=30, kde=True, color='skyblue')
plt.title('Price Distribution')
plt.show()

In [ ]:
contineous_features = ["Duration"]
discrete_features = []
ordinal_features = ["Dep_Time", "Arrival_Time", "Total_Stops", "Date_of_Journey"]
nominal_features = ["Airline", "Source", "Destination", "Route", "Additional_Info"]

# Preprocessing

## Remove Redundant Columns

In [ ]:
# Route not needed since it's captured in Total_Stops and Arrival and Departure features
# Remove Arrival_Time as this is capture from Departure Time and Duration
data.drop(["Route", "Arrival_Time"], axis=1, inplace=True)
ordinal_features.remove("Arrival_Time")
nominal_features.remove("Route")

## Data with duplicate values because of casing

In [ ]:
categorical_columns = ["Airline", "Source", "Destination", "Total_Stops", "Additional_Info"]
for col in categorical_columns:
    data[col] = data[col].str.lower().str.strip()

## Change Duplicate Records to the same

In [ ]:
data.loc[data['Airline'] == 'jet airways business', 'Additional_Info'] = 'business class'
data.loc[data['Airline'] == 'jet airways business', 'Airline'] = 'jet airways'
data.loc[data['Airline'] == "multiple carriers premium economy", 'Additional_Info'] = 'premium economy class'
data.loc[data['Airline'] == "multiple carriers premium economy", 'Airline'] = 'multiple carriers'
data.loc[data['Airline'] == 'vistara premium economy', 'Additional_Info'] = 'premium economy class'
data.loc[data['Airline'] == 'vistara premium economy', 'Airline'] = 'vistara'
data.loc[data['Destination'] == 'new delhi', 'Destination'] = 'delhi'

## Impute Missing Values

In [ ]:
data['Total_Stops'] = data['Total_Stops'].fillna(data['Total_Stops'].mode()[0])

## Encode Ordinal Data

In [ ]:
encoder = OrdinalEncoder(categories=[["non-stop", "1 stop", "2 stops", "3 stops", "4 stops"]])
data['Total_Stops'] = encoder.fit_transform(data[['Total_Stops']])
data['Total_Stops'] = data['Total_Stops'].astype(int)

## Date Conversion

In [ ]:
def extract_duration(duration_str):
    """Extracts hours and minutes from a duration string.

    Args:
        duration_str: The duration string in the format "HHh MMm" or "HHh" or "MMm".

    Returns:
        A tuple containing the number of hours and minutes.
    """
    hours = 0
    minutes = 0
    if 'h' in duration_str:
        hours_str, rest = duration_str.split('h')
        hours = int(hours_str)
    if 'm' in duration_str:
        if 'h' in duration_str:
            minutes_str = rest.strip('m') 
        else:
            minutes_str = duration_str.strip('m')
        minutes = int(minutes_str)
    return hours, minutes

In [ ]:
data['Date_of_Journey'] = pd.to_datetime(data['Date_of_Journey'], format='%d/%m/%Y')
# Extract day, month, year
data['Journey_day'] = data['Date_of_Journey'].dt.day
data['Journey_month'] = data['Date_of_Journey'].dt.month
data['Journey_year'] = data['Date_of_Journey'].dt.year

ordinal_features.extend(["Journey_day", "Journey_month", "Journey_year"])

# Extract day of the week
data['Journey_day_of_week'] = data['Date_of_Journey'].dt.day_name() 
nominal_features.extend(['Journey_day_of_week'])

data['Dep_Time'] = pd.to_datetime(data['Dep_Time'], format='%H:%M')

# Extract hour and minute from 'Dep_Time'
data['Dep_hour'] = pd.to_datetime(data['Dep_Time']).dt.hour
data['Dep_min'] = pd.to_datetime(data['Dep_Time']).dt.minute
ordinal_features.extend(['Dep_hour', 'Dep_min'])

# Extract hours and minutes from 'Duration'
data['Duration_hours'], data['Duration_mins'] = zip(*data['Duration'].apply(extract_duration))

# Convert 'Duration_hours' and 'Duration_mins' to numeric
data['Duration_hours'] = data['Duration_hours'].astype(int)
data['Duration_mins'] = data['Duration_mins'].astype(int)
contineous_features.extend(['Duration_hours', 'Duration_mins'])
    
# Calculate total duration in minutes
data['Duration_total_mins'] = data['Duration_hours'] * 60 + data['Duration_mins'] 
contineous_features.extend(['Duration_total_mins'])

# Drop original columns
data.drop(['Date_of_Journey', 'Dep_Time', 'Duration', 'Duration_hours', 'Duration_mins'], axis=1, inplace=True) 
ordinal_features.remove('Date_of_Journey')
ordinal_features.remove("Dep_Time")
contineous_features.remove('Duration')
contineous_features.remove("Duration_hours")
contineous_features.remove('Duration_mins')

### Check the distribution of years

In [ ]:
data['Journey_year'].value_counts()

### Check the distribution of Duration of Total Minutes

In [ ]:
# Duration Total Minutes Distribution
plt.figure(figsize=(8, 5))
sns.histplot(data['Duration_total_mins'], bins=30, kde=True, color='skyblue')
plt.title('Duration Total Minutes Distribution')
plt.show()

## Encoding

In [ ]:
data = pd.get_dummies(data, columns=nominal_features, drop_first=True)

In [ ]:
bool_columns = data.select_dtypes(include=["bool"]).columns
data[bool_columns] = data[bool_columns].astype("int")

## Check for Homoscedasticity

In [ ]:
numeric_features = contineous_features + discrete_features + ordinal_features
for feature in numeric_features:
    plt.figure(figsize=(8, 6))
    sns.regplot(x=feature, y='Price', data=data, scatter_kws={'s': 50}, line_kws={'color': 'red', 'lw': 2})
    plt.title(f'{feature} vs Price with Regression Line')
    plt.xlabel(feature)
    plt.ylabel('Price')
    plt.show()

## Normalization and standardization

In [ ]:
scaler = RobustScaler()
data['Duration_total_mins'] = scaler.fit_transform(data[['Duration_total_mins']])

In [ ]:
selected_features = [feature for feature in ordinal_features if feature != "total_stops"]
ordinalEncoder = OrdinalEncoder()
standardScalar = StandardScaler()
data[selected_features] = ordinalEncoder.fit_transform(data[selected_features])
data[selected_features] = standardScalar.fit_transform(data[selected_features])

In [ ]:
data[['Total_Stops', 'Journey_day', 'Journey_month', 'Journey_year', 'Dep_hour', 'Dep_min', 'Duration_total_mins']] = data[['Total_Stops', 'Journey_day', 'Journey_month', 'Journey_year', 'Dep_hour', 'Dep_min', 'Duration_total_mins']].astype(int)

## Feature Engineering

In [ ]:
X = data.drop("Price", axis=1)
Y  = data["Price"]
x = sm.add_constant(X)
est = sm.OLS(Y, x).fit()
print(est.summary())

In [ ]:
X = data.drop(columns=['Price']) 
y = data['Price'] 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model Evaluation

## Random Forest Regressor

### Training

In [ ]:
# Define the parameter distribution for Random Forest Regressor
pca = PCA()
param_dist = {
    'pca__n_components': [None, 0.9, 0.95, 0.99],  # PCA number of components to test (None for no reduction)
     'rf__n_estimators': randint(100, 500),  # Use a higher number of trees
    'rf__max_features': ['sqrt'],  # Limit the number of features to consider
    'rf__max_depth': randint(5, 30),  # Reduce max depth
    'rf__min_samples_split': randint(5, 15),  # More samples to split
    'rf__min_samples_leaf': randint(3, 15),  # More samples per leaf
    'rf__bootstrap': [True]  # Use bootstrap sampling
}

# Initialize Random Forest Regressor (without specifying hyperparameters)
rf = RandomForestRegressor(random_state=42)
pipeline = Pipeline([
    ('pca', pca),  # Apply PCA for dimensionality reduction
    ('rf', rf)  # Apply Random Forest Regressor
])

# Initialize RandomizedSearchCV with the parameter distribution
random_search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_dist,
    n_iter=150,  # Number of different hyperparameter combinations to sample
    cv=kfold,  # 5-fold cross-validation
    scoring='neg_mean_squared_error',  # Use MSE as the scoring metric
    verbose=1,
    random_state=42,  # Fix random seed for reproducibility
    n_jobs=-1  # Use all cores for faster computation
)

# Train the model using RandomizedSearchCV
random_search.fit(X_train, y_train)

# Get the best model from the random search
best_model = random_search.best_estimator_
best_params = random_search.best_params_
best_score = random_search.best_score_

print("Best Param: ", best_params)
print("Best Score: ", best_score)

### Prediction

In [ ]:
y_pred = best_model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
rmse = root_mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Model Performance
print("\nRandom Forest Model Performance (RandomizedSearchCV):")
print("Mean Squared Error: ", mse)
print("Root Mean Squared Error: ", rmse)
print(f"R-Squared Value: {r2:.2f}")

### Visuals 

In [ ]:
visualizer = ResidualsPlot(best_model)
visualizer.fit(X_train, y_train)  # Fit the training data to the visualizer
visualizer.score(X_test, y_test)  # Evaluate the model on the test data
visualizer.show()                 # Finalize and render the plot

## SVC Regressor

In [ ]:
# Define the SVR model
svr = SVR()
pca = PCA()
# Define the parameter grid for SVR
param_grid = {
    'pca__n_components': [None, 0.9, 0.95, 0.99],  # PCA number of components to test (None for no reduction)
    'svr__C': [0.1, 1.0, 10.0, 100.0],           # Regularization parameter
    'svr__epsilon': [0.01, 0.1, 0.5, 1.0],      # Margin of tolerance
    'svr__kernel': ['linear', 'rbf', 'poly'],    # Kernel types
    'svr__degree': [2, 3, 4],                   # Degree for polynomial kernel
    'svr__gamma': ['scale', 'auto']
}

pipeline = Pipeline([
    ('pca', pca),  # Apply PCA for dimensionality reduction
    ('svr', svr)  # Apply Random Forest Regressor
])


grid_search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_grid,
    n_iter=150,
    cv=kfold,  # 10-fold cross-validation
    scoring='neg_mean_squared_error',  # Use MSE as the metric
    verbose=1,
    random_state=42,  # For reproducibility
)

grid_search.fit(X_train, y_train)
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters: ", best_params)
print("Best neg_mean_squared_error: ", best_score)

## Predict

In [ ]:
# Predict on test set
y_pred = best_model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
rmse = root_mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
# Model Performance
print("\nLinear Regression Performance:")
print(f"Mean Squared Error: {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"R-Squared Value: {r2:.2f}")

### Visualization

In [ ]:
visualizer = ResidualsPlot(best_model)
visualizer.fit(X_train, y_train)  # Fit the training data to the visualizer
visualizer.score(X_test, y_test)  # Evaluate the model on the test data
visualizer.show()        

## XgBoost

### Training

In [ ]:
xgb = XGBRegressor()
pca = PCA()
param_grid = {
    'pca__n_components': [None, 0.9, 0.95, 0.99],  # PCA number of components to test (None for no reduction)
    'xgb__max_depth': range(4, 10, 2),
    'xgb__min_child_weight': range(3, 6, 2),
    'xgb__gamma': [i / 10.0 for i in range(2, 5)],
    'xgb__subsample': [i / 10.0 for i in range(7, 10)],
    'xgb__colsample_bytree': [i / 10.0 for i in range(4, 10)],
    'xgb__reg_alpha': [0, 0.001, 0.01]
}

pipeline = Pipeline([
    ('pca', pca),  # Apply PCA for dimensionality reduction
    ('xgb', xgb)  # Apply Random Forest Regressor
])

random_search = RandomizedSearchCV(estimator=pipeline, param_distributions=param_grid,
                                   n_iter=150, scoring='neg_mean_squared_error',
                                   cv=kfold, verbose=1, n_jobs=-1, random_state=42)

random_search.fit(X_train, y_train)
best_params = random_search.best_params_
best_model = random_search.best_estimator_
best_score = random_search.best_score_
print("Best param: ", best_params)
print("Best Score: ", best_score)


### Prediction

In [ ]:
y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = root_mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
# Model Performance
print("\nLinear Regression Performance:")
print(f"Mean Squared Error: {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"R-Squared Value: {r2:.2f}")

### Visualization

In [ ]:
visualizer = ResidualsPlot(best_model)
visualizer.fit(X_train, y_train)  # Fit the training data to the visualizer
visualizer.score(X_test, y_test)  # Evaluate the model on the test data
visualizer.show()                 # Finalize and render the plot